In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras

In [2]:
housing = fetch_california_housing()
X_train, X_test, y_train, y_test = train_test_split(housing.data, housing.target)

In [3]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

###  Passing the features through deep and wide channels
#### This allows the neural network to learn both deep patterns (using deep path) and simple rules (through the short path)
#### In contrast, a regular MLP forces all the data to flow through the full stack of layers; thus, simple patterns in the data may end up being distorted by this sequence of transformations

In [4]:
input_ = keras.layers.Input(shape=X_train.shape[1:])
hidden = keras.layers.Dense(30, activation="relu")(input_)
hidden = keras.layers.Dense(30, activation="relu")(hidden)
concat = keras.layers.Concatenate()([input_, hidden])
output = keras.layers.Dense(1)(concat)
model = keras.Model(inputs=[input_], outputs=[output])

model.compile(loss="mean_squared_error", optimizer="sgd")
model.summary()

history = model.fit(X_train, y_train, epochs=20, validation_split=0.2)

mse_test = model.evaluate(X_test, y_test)

X_new = X_test[:3] #pretend these are new instances to do some testing
y_pred = model.predict(X_new)
print(y_test[:3], y_pred)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 8)]          0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 30)           270         input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 30)           930         dense[0][0]                      
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 38)           0           input_1[0][0]                    
                                                                 dense_1[0][0]                

### Handling multiple inputs - one set of features through the deep channel and the other through wide channel (possibly overlapping set of features)

In [5]:
input_A = keras.layers.Input(shape=[5], name="wide_input")
input_B = keras.layers.Input(shape=[6], name="deep_input")
hidden = keras.layers.Dense(30, activation="relu")(input_B)
hidden = keras.layers.Dense(30, activation="relu")(hidden)
concat = keras.layers.Concatenate()([input_A, hidden])
output = keras.layers.Dense(1)(concat)
model = keras.Model(inputs=[input_A, input_B], outputs=[output])

model.compile(loss="mean_squared_error", optimizer="sgd")
model.summary()

history = model.fit((X_train[:, :5], X_train[:, 2:]), y_train, epochs=20, validation_split=0.2)

mse_test = model.evaluate((X_test[:, :5], X_test[:, 2:]), y_test)

X_new = X_test[:3] #pretend these are new instances to do some testing
y_pred = model.predict((X_new[:, :5], X_new[:, 2:]))
print(y_test[:3], y_pred)

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
deep_input (InputLayer)         [(None, 6)]          0                                            
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 30)           210         deep_input[0][0]                 
__________________________________________________________________________________________________
wide_input (InputLayer)         [(None, 5)]          0                                            
__________________________________________________________________________________________________
dense_4 (Dense)                 (None, 30)           930         dense_3[0][0]                    
____________________________________________________________________________________________

### Handling multiple outputs; In this case to add an auxillary output for regularization
#### Each output needs its own loss function. So, we pass a list of losses
#### By default, keras will compute all these losses and simply add them up to get the final loss used for training. We care musch more about the main output then about the aux output. So, we give more weight to output's loss than aux output's loss

In [6]:
input_A = keras.layers.Input(shape=[5], name="wide_input")
input_B = keras.layers.Input(shape=[6], name="deep_input")
hidden = keras.layers.Dense(30, activation="relu")(input_B)
hidden = keras.layers.Dense(30, activation="relu")(hidden)
concat = keras.layers.Concatenate()([input_A, hidden])
output = keras.layers.Dense(1, name="main_output")(concat)
aux_output = keras.layers.Dense(1, name="aux_output")(hidden)
model = keras.Model(inputs=[input_A, input_B], outputs=[output, aux_output])

model.compile(loss=["mse", "mse"], loss_weights=[0.9, 0.1], optimizer="sgd")
model.summary()

history = model.fit((X_train[:, :5], X_train[:, 2:]), (y_train, y_train), epochs=20, validation_split=0.2)

mse_test = model.evaluate((X_test[:, :5], X_test[:, 2:]), (y_test, y_test))

X_new = X_test[:3] #pretend these are new instances to do some testing
y_pred = model.predict((X_new[:, :5], X_new[:, 2:]))
print(y_test[:3], y_pred)

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
deep_input (InputLayer)         [(None, 6)]          0                                            
__________________________________________________________________________________________________
dense_6 (Dense)                 (None, 30)           210         deep_input[0][0]                 
__________________________________________________________________________________________________
wide_input (InputLayer)         [(None, 5)]          0                                            
__________________________________________________________________________________________________
dense_7 (Dense)                 (None, 30)           930         dense_6[0][0]                    
____________________________________________________________________________________________